In [1]:
import sys
import mygene
import re
import io
import requests
import logging
import urllib.request as url
import numpy as np
import pandas as pd
import ipywidgets as widgets
import warnings
import bokeh.palettes
from operator import itemgetter
from intermine.webservice import Service
from IPython.display import display, clear_output, HTML
from termcolor import colored
from bokeh.io import show, output_notebook
from bokeh.models import ColorBar, ColumnDataSource, CategoricalColorMapper
from bokeh.plotting import figure
from bokeh.transform import transform

output_notebook(hide_banner=True)
sys.path.append('..')
warnings.simplefilter("ignore", UserWarning)
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

impc_api_url = "https://www.ebi.ac.uk/mi/impc/bulkdata-api"
impc_api_search_url = f"{impc_api_url}/genes"
impc_api_gene_bundle_url = f"{impc_api_url}/geneBundles"
systems = ["immune system phenotype", "integument phenotype", "adipose tissue phenotype",
               "hearing/vestibular/ear phenotype", "hematopoietic system phenotype", "craniofacial phenotype",
               "cardiovascular system phenotype", "renal/urinary system phenotype", "homeostasis/metabolism phenotype",
               "pigmentation phenotype", "limbs/digits/tail phenotype", "nervous system phenotype",
               "vision/eye phenotype", "liver/biliary system phenotype", "respiratory system phenotype",
               "behavior/neurological phenotype", "skeleton phenotype", "mortality/aging",
               "reproductive system phenotype", "endocrine/exocrine gland phenotype",
               "growth/size/body region phenotype", "embryo phenotype", "muscle phenotype",
               "digestive/alimentary phenotype"]

EXPLAINATION = """\
<div class="app-sidebar">

<p>Submit a list of genes MGI ids or genes symbol separted by a comma, semicolumn, tab or newline. The program will create an heatmap representing the number of phenotypes for each gene contained in an <a href="https://www.mousephenotype.org/help/data-visualization/gene-pages/phenogrid/">
IMPC major category</a>.<br> Using the slider, adjust the treshold to set the minimum count to be displayed in the heatmap. Please note that genes without phenotypes in any category will not be displayed.</p>
</div>
"""

In [2]:
class App:
    def __init__(self):
        self._plot_container = widgets.Output()
        self._load_button, load_box = self._create_load_button()
        self._reset_button = self._create_reset_button()
        
        self.container = widgets.VBox([
            widgets.VBox([load_box,widgets.HBox([self._load_button, self._reset_button])])], layout = widgets.Layout(align_items='center'))
        self.final_container = widgets.VBox([
            widgets.HTML(('<h1><em><b>Gene vs IMPC phenotype category heatmap</b></em></h1>'), 
                         layout=widgets.Layout(margin='0 0 5em 0')),
            widgets.HTML(EXPLAINATION,layout=widgets.Layout(align_items='center')),
            self.container,self._plot_container], layout = widgets.Layout(align_items='center', flex='3 0 auto'))
    
    
    
    #Data loading and conversion from symbols to MGI ids
    
    def _create_load_button(self):
        load_label = widgets.Label('List of genes:')
        self.list_area = widgets.Textarea(placeholder="E.g: Pax6,MGI:00001,...")
        load_b = widgets.Button(description="Submit", tooltip="submit your gene list", disabled=True)
        load_b.on_click(self._on_change_l)
        self.list_area.observe(self._on_change_text, names =["value"])
        sub_box = widgets.HBox([load_label,self.list_area])
        return (load_b, sub_box)
    
    def _create_reset_button(self):
        reset_b = widgets.Button(description="Reset", tooltip="reset the environment to generate a new plot", disabled = True)
        reset_b.on_click(self._on_click_r)
        return (reset_b)
        
    def _on_change_text(self,_):
        if (self.list_area.value == "") | (self.list_area.value.isspace()):
            self._load_button.disabled=True
        else:
            self._load_button.disabled=False
        
    def _on_click_r(self, _):    
        final_list = []
        sym_list = []
        self.data = []
        self.not_mapped = []
        self.not_valid = []
        self.list_area.value=""
        self._reset_button.disabled=True
        self._load_button.disabled=True
        self._plot_container.clear_output(wait=False)
        remove = self.container.children[-1]
        self.container.children = self.container.children[:-1]
        remove.close()
        
    def _on_change_l(self, _):
        with self._plot_container:
            loading = widgets.HTML(value='<i class=\"fa fa-spinner fa-spin fa-5x fa-fw\" style="color: #f09f14;"></i>')
            display(loading)
            self._load_button.disabled=True
            self.data = self.map_gene()
            self.not_valid = []
            self.not_mapped = []
            self.df = pd.DataFrame(data='', index=self.data, columns=systems)
            self.genes = [item.strip() for item in re.split(r',|,\s|;|;\s|\n|\t',self.list_area.value)]
            final_list = []
            sym_list = []
            self.not_mapped = []
            self.not_valid = []
            for i in self.genes:
                if 'MGI:' in i:
                    final_list.append(i)
                else:
                    sym_list.append(i)

            if len(sym_list) != 0:
                mapping = pd.read_csv('../../gene_mapping.csv', index_col=1, dtype="string").T.to_dict()
                res = []
                for i in sym_list:
                    try:
                        res.append(list(mapping[i].values())[0])
                    except (KeyError) as e:
                        self.not_mapped.append(i)
                if len(res) == 0:
                    print("\033[91m Error: it was not possible to map the input.")
                    self._reset_button.disabled=False
                else:
                    final_list.extend(res)

            self.df = pd.DataFrame(data='', index=final_list, columns=systems)

            for i in self.df.index:
                gene_url = f"{impc_api_search_url}/{i}"
                try:
                    gene_data = requests.get(gene_url).json()

                except (ValueError) as e:
                    self.not_valid.append(i)

                else:
                    sigMp = gene_data['significantMpTerms']
                    if sigMp is None:
                        continue
                    l = []
                    for j in sigMp:
                        if j['mpTermId'] is None:
                            continue
                        l.append(j['mpTermId'])
                        for k in j['topLevelAncestors']:
                            if self.df.loc[i, k['mpTermName']] == '':
                                self.df.loc[i, k['mpTermName']] = l
                            else:
                                self.df.loc[i, k['mpTermName']].extend(l)
                        l=[]

            if len(self.not_valid) != 0:
                self.df = self.df[~self.df.index.isin(self.not_valid)]
            if len(self.df) != 0:    
                self.df.index = self.revert_to_symbols(self.df.index)
                self._create_matrices()
            else:
                print("\033[91m Error: no valid MGI ids found.")
                self._reset_button.disabled=False
        
    #Matrices construction
    
    def _create_matrices(self):
        
        self.c_mat = pd.DataFrame(index=self.df.index, columns=self.df.columns)
        
        for i in self.df.index:
            for j in self.df.columns:
                x = re.sub("[\[\]']","",str(self.df.loc[i,j]))
                self.c_mat.loc[i,j] = (len(self.df.loc[i,j]), x)
                del(x)
                
        
        self.c_mat.index.name = 'MGI_id'
        self.c_mat.columns.name = 'phen_sys'
        self._slider, self.slider_box = self._create_slider()
        self.container.children = tuple(list(self.container.children) + [self.slider_box])
        self._update_app()
    
    #Creation of the thrashold slider
    
    def _create_slider(self):
        slider_label = widgets.Label('Threshold: ')
        slider = widgets.IntSlider(value=0, min=0, max = 0, step=1, orintation='horizontal', readout=True, readout_format="d")
        slider.observe(self._on_change, names=['value'])
        slider_box = widgets.HBox([slider_label,slider])
        return (slider, slider_box)
    
    def _on_change(self, _):
        self._update_app()
        
    #Slider value interactivness
    
    def _update_app(self):
        self._slider.max = max(list(self.c_mat.max()), key=itemgetter(0))[0]
        threshold = self._slider.value
        try:
            with self._plot_container:
                p = self._create_plot(threshold)
                self._reset_button.disabled=False
                self._plot_container.clear_output(wait=True)
                show(p, notebook_handle=True)
                if len(self.not_mapped) != 0:
                    print("\033[93m Warning: some input were not mapped: " + str(self.not_mapped))
                if len(self.not_valid) != 0:
                    print("\033[93m Warning: some MGI ids were not found: " + str(self.not_valid))
        except (NameError,AttributeError) as e:
            pass    
    
    #Plot generation
    
    def _create_plot(self, threshold):
        tem_mat= self.c_mat.copy()
        rem=[]
        if threshold != 0:
            for i in tem_mat.index:
                if tem_mat.loc[i].max()[0] < threshold:
                    rem.append(i)
            tem_mat.drop(rem,inplace=True,axis=0)

        #Create a custom palette and add a specific mapper to map color with values, we are converting them to strings to create a categorical color mapper to include only the
        #values that we have in the matrix and retrieve a better representation
        
        tmp = tem_mat.stack(dropna=False).rename("value").reset_index()
        tmp["pheno"] = ""
        for i in range(0,len(tmp)):
            tmp.pheno[i] = tmp.value[i][1]
            tmp.value[i] = tmp.value[i][0]
        fact= tmp.value.unique()
        fact.sort()
        fact = fact.astype(str)
        tmp.value = tmp.value.astype(str)
        tmp.pheno = tmp.pheno.astype(str)

        mapper = CategoricalColorMapper(palette=bokeh.palettes.inferno(len(tmp.value.unique())), factors= fact, nan_color = 'gray')

        #Define a figure
        p = figure(
            plot_width=1280,
            plot_height=800,
            x_range=list(tmp.phen_sys.drop_duplicates()),
            y_range=list(tmp.MGI_id.drop_duplicates()[::-1]),
            tooltips=[('Phenotype system','@phen_sys'),('Gene','@MGI_id'),('Number of phenotypes','@value'), ('Phenotype list: ','@pheno')],
            x_axis_location="above",
            output_backend="webgl",
            toolbar_location="right",
            tools="pan,wheel_zoom,box_zoom,reset,save")

        #Create rectangles for heatmap
        p.rect(
            x="phen_sys",
            y="MGI_id",
            width=1,
            height=1,
            source=ColumnDataSource(tmp),
            fill_color=transform('value', mapper))
        p.xaxis.major_label_orientation = 45

        #Add legend
        color_bar = ColorBar(
        color_mapper=mapper,
        label_standoff=6,
        border_line_color=None)
        p.add_layout(color_bar, 'right')
        return(p)
    
    def map_gene(self):
        tmp = [item.strip() for item in re.split(r',|,\s|;|;\s|\n|\t|\s',self.list_area.value)]
        final_list = []
        # symbol for symbols, mgi for MGI : https://docs.mygene.info/en/latest/doc/query_service.html#available-fields
        mg = mygene.MyGeneInfo()
        ginfo = mg.querymany(tmp, scopes='symbol', fields="symbol,MGI", species='mouse')
        empty = True
        discarded = []
        for i in ginfo:
            try:
                final_list.append(i['MGI'])
                empty = False
            except KeyError:
                if "MGI:" in i['query']:
                    final_list.append(i['query'])
                else:
                    discarded.append(i['query'])
        if empty and len(final_list) == 0:
            stop_err("Error: it was not possible to map the input.")
        elif empty:
            print("Warning: it was not possible to map any of the symbol ids. Only MGI ids will be used.")
        elif len(discarded) != 0:
            print("Warning: it was not possible to map these elements: " + ",".join(discarded) + "\n")

        return(final_list)
    
    def revert_to_symbols(self,mgi_list):
        back = []
        mg = mygene.MyGeneInfo()
        ginfo = mg.querymany(mgi_list, scopes='mgi', fields="symbol", species='mouse')

        for j in ginfo:
            try:
                back.append(j['symbol'])
            except KeyError:
                print("It was not possible to map " + j['query'])
                back.append(j['query'])
                
        return(back)

app = App()
app.final_container